# 📃 Solution for Exercise M1.03

The goal of this exercise is to compare the performance of our classifier in
the previous notebook (roughly 81% accuracy with `LogisticRegression`) to
some simple baseline classifiers. The simplest baseline classifier is one
that always predicts the same class, irrespective of the input data.

- What would be the score of a model that always predicts `' >50K'`?
- What would be the score of a model that always predicts `' <=50K'`?
- Is 81% or 82% accuracy a good score for this problem?

Use a `DummyClassifier` and do a train-test split to evaluate
its accuracy on the test set. This
[link](https://scikit-learn.org/stable/modules/model_evaluation.html#dummy-estimators)
shows a few examples of how to evaluate the statistical performance of these
baseline models.

In [1]:
import pandas as pd

adult_census = pd.read_csv("../datasets/adult-census.csv")

We will first split our dataset to have the target separated from the data
used to train our predictive model.

In [2]:
target_name = "class"
target = adult_census[target_name]
data = adult_census.drop(columns=target_name)

We start by selecting only the numerical columns as seen in the previous
notebook.

In [3]:
numerical_columns = [
    "age", "capital-gain", "capital-loss", "hours-per-week"]

data_numeric = data[numerical_columns]

Next, let's split the data and target into a train and test set.

In [4]:
from sklearn.model_selection import train_test_split

data_numeric_train, data_numeric_test, target_train, target_test = \
    train_test_split(data_numeric, target, random_state=42)

We will first create a dummy classifier which will always predict the
high revenue class, i.e. `" >50K"`, and check the statistical
performance.

In [5]:
from sklearn.dummy import DummyClassifier

class_to_predict = " >50K"
high_revenue_clf = DummyClassifier(strategy="constant",
                                   constant=class_to_predict)
high_revenue_clf.fit(data_numeric_train, target_train)
score = high_revenue_clf.score(data_numeric_test, target_test)
print(f"Accuracy of a model predicting only high revenue: {score:.3f}")

Accuracy of a model predicting only high revenue: 0.234


We clearly see that the score is below 0.5 which might be surprising at
first. We will now check the statistical performance of a model which always
predict the low revenue class, i.e. `" <=50K"`.

In [6]:
class_to_predict = " <=50K"
low_revenue_clf = DummyClassifier(strategy="constant",
                                  constant=class_to_predict)
low_revenue_clf.fit(data_numeric_train, target_train)
score = low_revenue_clf.score(data_numeric_test, target_test)
print(f"Accuracy of a model predicting only low revenue: {score:.3f}")

Accuracy of a model predicting only low revenue: 0.766


We observe that this model has an accuracy higher than 0.5. This is due to
the fact that we have 3/4 of the target belonging to low-revenue class.

Therefore, any predictive model giving results below this dummy classifier
will not be helpful.

In [7]:
adult_census["class"].value_counts()

 <=50K    37155
 >50K     11687
Name: class, dtype: int64

In [8]:
(target == " <=50K").mean()

0.7607182343065395

In practice, we could have the strategy `"most_frequent"` to predict the
class that appears the most in the training target.

In [9]:
most_freq_revenue_clf = DummyClassifier(strategy="most_frequent")
most_freq_revenue_clf.fit(data_numeric_train, target_train)
score = most_freq_revenue_clf.score(data_numeric_test, target_test)
print(f"Accuracy of a model predicting the most frequent class: {score:.3f}")

Accuracy of a model predicting the most frequent class: 0.766


So the `LogisticRegression` accuracy (roughly 81%) seems better than the
`DummyClassifier` accuracy (roughly 76%). In a way it is a bit reassuring,
using a machine learning model gives you a better performance than always
predicting the majority class, i.e. the low income class `" <=50K"`.